In [2]:
# |default_exp modelsconv


In [3]:
#| export

import torch
from torch import nn, einsum
from einops import rearrange, repeat
from rotary_embedding_torch import RotaryEmbedding, apply_rotary_emb

import torch.nn.functional as F
import torch.utils.checkpoint as checkpoint
import math
from timm.models.layers import drop_path, to_2tuple, trunc_normal_
from torch_sparse import SparseTensor, matmul
from torch_geometric.utils import degree
from torch_geometric.data import Data, Batch
import numpy as np
from torch_geometric.utils import to_dense_batch
from x_transformers import ContinuousTransformerWrapper, Encoder, TransformerWrapper
from torch_geometric.nn import GATConv, GCNConv


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
import matplotlib.pyplot as plt

In [5]:
#| export
def good_luck():
    return True

In [6]:
# | export
class conv_block(nn.Module):
    def __init__(self, ch_in, ch_out):
        super(conv_block, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True),
            nn.Conv2d(ch_out, ch_out, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class up_conv(nn.Module):
    def __init__(self, ch_in, ch_out):
        super(up_conv, self).__init__()
        self.up = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv2d(ch_in, ch_out, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(ch_out),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x = self.up(x)
        return x


class U_Net(nn.Module):
    def __init__(self, img_ch=3, output_ch=1, CH_FOLD2=1):
        super(U_Net, self).__init__()

        self.Maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.Conv1 = conv_block(ch_in=img_ch, ch_out=int(32 * CH_FOLD2))
        self.Conv2 = conv_block(ch_in=int(32 * CH_FOLD2), ch_out=int(64 * CH_FOLD2))
        self.Conv3 = conv_block(ch_in=int(64 * CH_FOLD2), ch_out=int(128 * CH_FOLD2))
        self.Conv4 = conv_block(ch_in=int(128 * CH_FOLD2), ch_out=int(256 * CH_FOLD2))
        self.Conv5 = conv_block(ch_in=int(256 * CH_FOLD2), ch_out=int(512 * CH_FOLD2))

        self.Up5 = up_conv(ch_in=int(512 * CH_FOLD2), ch_out=int(256 * CH_FOLD2))
        self.Up_conv5 = conv_block(
            ch_in=int(512 * CH_FOLD2), ch_out=int(256 * CH_FOLD2)
        )

        self.Up4 = up_conv(ch_in=int(256 * CH_FOLD2), ch_out=int(128 * CH_FOLD2))
        self.Up_conv4 = conv_block(
            ch_in=int(256 * CH_FOLD2), ch_out=int(128 * CH_FOLD2)
        )

        self.Up3 = up_conv(ch_in=int(128 * CH_FOLD2), ch_out=int(64 * CH_FOLD2))
        self.Up_conv3 = conv_block(ch_in=int(128 * CH_FOLD2), ch_out=int(64 * CH_FOLD2))

        self.Up2 = up_conv(ch_in=int(64 * CH_FOLD2), ch_out=int(32 * CH_FOLD2))
        self.Up_conv2 = conv_block(ch_in=int(64 * CH_FOLD2), ch_out=int(32 * CH_FOLD2))

        self.Conv_1x1 = nn.Conv2d(
            int(32 * CH_FOLD2), output_ch, kernel_size=1, stride=1, padding=0
        )

    def forward(self, x):
        # encoding path
        x1 = self.Conv1(x)

        x2 = self.Maxpool(x1)
        x2 = self.Conv2(x2)

        x3 = self.Maxpool(x2)
        x3 = self.Conv3(x3)

        x4 = self.Maxpool(x3)
        x4 = self.Conv4(x4)

        x5 = self.Maxpool(x4)
        x5 = self.Conv5(x5)

        # decoding + concat path
        d5 = self.Up5(x5)
        d5 = torch.cat((x4, d5), dim=1)

        d5 = self.Up_conv5(d5)

        d4 = self.Up4(d5)
        d4 = torch.cat((x3, d4), dim=1)
        d4 = self.Up_conv4(d4)

        d3 = self.Up3(d4)
        d3 = torch.cat((x2, d3), dim=1)
        d3 = self.Up_conv3(d3)

        d2 = self.Up2(d3)
        d2 = torch.cat((x1, d2), dim=1)
        d2 = self.Up_conv2(d2)

        d1 = self.Conv_1x1(d2)
        d1 = d1.squeeze(1)
        out = torch.transpose(d1, -1, -2) * d1

        return out


class UnetWrapper2D(nn.Module):
    def __init__(self, md, output_chans=2):
        super().__init__()
        self.md = md
        self.output_chans = output_chans

    def do_forward(self, x, crop16, crop):
        out = torch.zeros(
            x.shape[0], self.output_chans, x.shape[2], x.shape[3], device=x.device
        )
        res = self.md(x[:, :, :crop16, :crop16])
        out[:, :, :crop, :crop] = res[:, :, :crop, :crop]
        return out

    def forward(self, xs, crop_to_16, crop_original, original_order):
        res = []
        for x, crop16, crop in zip(xs, crop_to_16, crop_original):
            res.append(self.do_forward(x, crop16, crop))
        return torch.cat(res)[original_order]


@torch.no_grad()
def generate_batches(tensor, nn_out):
    # Sort the tensor
    if tensor.unique().shape[0] == 1:
        return [nn_out], torch.arange(len(tensor), device=tensor.device)
    sorted_tensor, order = tensor.sort()
    nn_out = nn_out.index_select(0, order)

    # Find the change points
    diff = torch.cat([torch.tensor([1]), torch.diff(sorted_tensor)])
    change_indices = torch.where(diff != 0)[0]
    change_indices = torch.cat([change_indices, torch.tensor([len(tensor)])])
    b = [
        nn_out[change_indices[i] : change_indices[i + 1]]
        for i in range(len(change_indices) - 1)
    ]
    return b, order.argsort()


def make_pair_mask(seq, seq_len):
    encode_mask = torch.arange(seq.shape[1], device=seq.device).expand(
        seq.shape[:2]
    ) < seq_len.unsqueeze(1)
    pair_mask = encode_mask[:, None, :] * encode_mask[:, :, None]
    assert isinstance(pair_mask, torch.BoolTensor) or isinstance(
        pair_mask, torch.cuda.BoolTensor
    )
    return torch.bitwise_not(pair_mask)


class ScaledSinuEmbedding(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.scale = nn.Parameter(
            torch.ones(
                1,
            )
        )
        inv_freq = 1.0 / (10000 ** (torch.arange(0, dim, 2).float() / dim))
        self.register_buffer("inv_freq", inv_freq)

    def forward(self, x):
        n, device = x.shape[1], x.device
        t = torch.arange(n, device=device).type_as(self.inv_freq)
        sinu = einsum("i , j -> i j", t, self.inv_freq)
        emb = torch.cat((sinu.sin(), sinu.cos()), dim=-1)
        return emb * self.scale


class CustomEmbedding(nn.Module):
    def __init__(self, dim, vocab=4):
        super().__init__()
        self.embed_seq = nn.Embedding(vocab, dim)
        self.pos_enc = ScaledSinuEmbedding(dim)

    def forward(self, x):
        x = self.embed_seq(x)
        x = x + self.pos_enc(x)
        return x


class SeqToImage(nn.Module):
    def __init__(self, dim, vocab=4):
        super().__init__()
        self.embed_h = CustomEmbedding(dim=dim, vocab=vocab)
        self.embed_w = CustomEmbedding(dim=dim, vocab=vocab)
        self.norm = nn.LayerNorm(dim)

    def forward(self, seq, mask):
        seq_h = self.embed_h(seq)
        seq_w = self.embed_w(seq)
        x = seq_h.unsqueeze(1) + seq_w.unsqueeze(2)
        x = self.norm(x)
        x.masked_fill_(mask[:, :, :, None], 0.0)  # bs, h, w, dim
        x = x.permute(0, 3, 1, 2)  # bs, dim, h, w
        return x


class Attn_pool(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.conv = nn.Conv2d(n, n, 1)
        self.attn = nn.Conv2d(n, n, 1)

    def forward(self, x, key_padding_mask=None):
        emb = self.conv(x)
        attn = self.attn(x)

        # Apply the mask to attention scores before softmax
        if key_padding_mask is not None:
            attn = attn.masked_fill(key_padding_mask.unsqueeze(1), float("-inf"))
        # attn = torch.clamp(attn, min=-1e9, max=1e9)
        attn = attn.softmax(dim=-1)
        x = (emb * attn).sum(-1)
        return x, attn


class FeedForwardV5(nn.Module):
    def __init__(self, dim, hidden_dim, dropout=0.2, out=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, out),
        )

    def forward(self, x):
        return self.net(x)


class RnaModelConvV0(nn.Module):
    def __init__(self, embed_size, conv_out=8, vecob_size=4):
        super().__init__()
        self.seq_to_image = SeqToImage(embed_size, vocab=vecob_size)
        self.md = UnetWrapper2D(U_Net(embed_size, conv_out), conv_out)
        self.attnpool = Attn_pool(conv_out)
        self.out = FeedForwardV5(conv_out, conv_out, out=2)

    def forward(self, batch):
        L_seq = batch["mask"].sum(1)
        L0 = batch["mask"].shape[1]

        crop_to_original = L_seq.unique()  # unique lengths
        crop_to_16 = [
            ((i // 16) + 1) * 16 for i in crop_to_original
        ]  # for each unique length, find the nearest 16 miltip
        seq = batch["seq"][:, : crop_to_16[-1]]  # shortening to largest 16 multiple

        # make a square mask [bs, crop_to_16[-1], crop_to_16[-1]]  #crop_to_16[-1] is the largest 16 multiple
        square_mask = make_pair_mask(seq, L_seq)

        x = self.seq_to_image(seq, square_mask)
        x, idc = generate_batches(L_seq, x)
        x = self.md(x, crop_to_16, crop_to_original, idc)
        x, attn = self.attnpool(x, square_mask)
        x = self.out(x.permute(0, 2, 1))
        x = F.pad(x, (0, 0, 0, L0 - x.shape[1], 0, 0))
        return x

In [7]:
embed_size = 16
out_chan_unet = 32
md = RnaModelConvV0(embed_size=embed_size, conv_out=out_chan_unet).eval()
batch = torch.load("batch.pt")
with torch.no_grad():
    out = md(batch)

torch.Size([10, 32, 128, 128])
torch.Size([10, 32, 128, 128])
torch.Size([14, 32, 160, 160])
torch.Size([14, 32, 160, 160])
torch.Size([8, 32, 176, 176])
torch.Size([8, 32, 176, 176])


In [8]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()

In [9]:
out

tensor([[[-0.3950, -0.0395],
         [-0.3943, -0.0402],
         [-0.3931, -0.0413],
         ...,
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[-0.3950, -0.0395],
         [-0.3943, -0.0402],
         [-0.3931, -0.0413],
         ...,
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[-0.3950, -0.0395],
         [-0.3943, -0.0402],
         [-0.3931, -0.0413],
         ...,
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        ...,

        [[-0.3951, -0.0393],
         [-0.3944, -0.0401],
         [-0.3933, -0.0412],
         ...,
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[-0.3951, -0.0394],
         [-0.3944, -0.0401],
         [-0.3933, -0.0412],
         ...,
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[-0.3951, -0.0393],
       

In [11]:
U_Net(3, 2)(torch.rand(1, 3, 256, 256))

torch.Size([1, 2, 256, 256])
torch.Size([1, 2, 256, 256])


tensor([[[[ 6.9599e-02,  2.0665e-02,  3.2410e-02,  ...,  4.6228e-04,
           -1.1477e-01,  8.5805e-02],
          [ 2.0665e-02,  3.3128e-03,  2.3444e-02,  ...,  4.2860e-03,
            8.9023e-02, -1.6158e-02],
          [ 3.2410e-02,  2.3444e-02,  5.2872e-01,  ...,  7.1295e-02,
           -3.9363e-02,  1.4245e-01],
          ...,
          [ 4.6228e-04,  4.2860e-03,  7.1295e-02,  ...,  1.9310e-02,
            2.4313e-02,  1.9723e-02],
          [-1.1477e-01,  8.9023e-02, -3.9363e-02,  ...,  2.4313e-02,
            2.7535e-01, -6.4474e-02],
          [ 8.5805e-02, -1.6158e-02,  1.4245e-01,  ...,  1.9723e-02,
           -6.4474e-02,  3.3966e-02]],

         [[ 6.8054e-04,  2.6302e-02, -2.7471e-02,  ...,  6.0433e-02,
            1.6723e-01,  4.9745e-02],
          [ 2.6302e-02,  1.8257e-01,  8.9803e-02,  ...,  9.4198e-02,
            8.6957e-01, -1.5848e-02],
          [-2.7471e-02,  8.9803e-02,  5.0949e-02,  ...,  2.9446e-01,
            1.5864e-01, -9.6408e-02],
          ...,
     

In [15]:
 torch.transpose(torch.rand(1, 3, 256, 256), -1, -2).shape

torch.Size([1, 3, 256, 256])

In [86]:
x = torch.arange(4).view(1, 1, 2, 2) + 1
y = torch.arange(4).view(1, 1, 2, 2)


In [87]:
x

tensor([[[[1, 2],
          [3, 4]]]])

In [88]:
y

tensor([[[[0, 1],
          [2, 3]]]])

In [89]:
out = torch.transpose(x, -1, -2) * y

In [90]:
out[0][0]

tensor([[ 0,  3],
        [ 4, 12]])

In [91]:
torch.transpose(x, -1, -2)

tensor([[[[1, 3],
          [2, 4]]]])